### Train the model:

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import torch

# Load the pre-trained Xception model
base_model = tf.keras.applications.Xception(input_shape=(256, 256, 3),
                                            include_top=False,
                                            weights='imagenet')

# Unfreeze the base model
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

# Defining my ArcFace Function
class ArcFace(layers.Layer):
    def __init__(self, n_classes=300, s=64.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizer

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[0][-1], self.n_classes),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = tf.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        theta = tf.acos(tf.clip_by_value(logits, -1.0 + tf.keras.backend.epsilon(), 1.0 - tf.keras.backend.epsilon()))
        target_logits = tf.cos(theta + self.m)
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'regularizer': self.regularizer,
        })
        return config

# Add new layers
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Placeholder for label input
y = tf.keras.Input(shape=(300,))  # assuming you have 300 classes
output = ArcFace(n_classes=300)([x, y])
print(output)

# Define the new model
model = Model(inputs=[base_model.input, y], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Create a data generator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Custom data generator
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, directory, datagen, batch_size=32, target_size=(256, 256), class_mode='categorical'):
        self.generator = datagen.flow_from_directory(directory, target_size=target_size, batch_size=batch_size, class_mode=class_mode)
        
    def __len__(self):
        return len(self.generator)
    
    def __getitem__(self, index):
        x, y = self.generator[index]
        return [x, y], y

# Create custom data generators
train_generator = CustomDataGen('ocular_recognition/train', train_datagen)
val_generator = CustomDataGen('ocular_recognition/validation', val_datagen)

# Set up callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=20)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
os.makedirs('trained_models', exist_ok=True)
model_checkpoint = ModelCheckpoint('trained_models/Xception_best_model_ArcFace_Base.h5', save_best_only=True)

callbacks = [early_stop, reduce_lr, model_checkpoint]

# Fit the model on the batches generated by datagen.flow()
model.fit(train_generator,
          validation_data=val_generator,
          epochs=100,
          callbacks=callbacks)

KerasTensor(type_spec=TensorSpec(shape=(None, 300), dtype=tf.float32, name=None), name='arc_face/Softmax:0', description="created by layer 'arc_face'")
Found 5076 images belonging to 300 classes.
Found 1621 images belonging to 300 classes.
Epoch 1/100
159/159 [==============================] - 64s 378ms/step - loss: 36.7013 - accuracy: 0.0000e+00 - val_loss: 34.4819 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/100
159/159 [==============================] - 60s 374ms/step - loss: 31.3214 - accuracy: 0.0000e+00 - val_loss: 23.1279 - val_accuracy: 0.0234 - lr: 1.0000e-04
Epoch 3/100
159/159 [==============================] - 59s 371ms/step - loss: 24.2632 - accuracy: 0.0000e+00 - val_loss: 13.0898 - val_accuracy: 0.2307 - lr: 1.0000e-04
Epoch 4/100
159/159 [==============================] - 60s 373ms/step - loss: 18.3506 - accuracy: 0.0030 - val_loss: 7.3249 - val_accuracy: 0.4954 - lr: 1.0000e-04
Epoch 5/100
159/159 [==============================] - 60s 373ms/step - loss: 13.7849

### Evaluation:

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import os
import random

# Defining my ArcFace Function
class ArcFace(layers.Layer):
    def __init__(self, n_classes=300, s=64.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizer

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[0][-1], self.n_classes),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = tf.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        theta = tf.acos(tf.clip_by_value(logits, -1.0 + tf.keras.backend.epsilon(), 1.0 - tf.keras.backend.epsilon()))
        target_logits = tf.cos(theta + self.m)
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'regularizer': self.regularizer,
        })
        return config
    
# Load the saved model
# model = load_model('trained_models/Xception_best_model_ArcFace_2.h5', custom_objects={'ArcFace': ArcFace})

# Create a data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Custom data generator
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, directory, datagen, batch_size=32, target_size=(256, 256), class_mode='categorical'):
        self.generator = datagen.flow_from_directory(directory, target_size=target_size, batch_size=batch_size, class_mode=class_mode)
        
    def __len__(self):
        return len(self.generator)
    
    def __getitem__(self, index):
        x, y = self.generator[index]
        return [x, y], y

# Load images from the test directory
test_generator = CustomDataGen('ocular_recognition/test', test_datagen)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Found 1621 images belonging to 300 classes.
51/51 [==============================] - 5s 91ms/step - loss: 0.0413 - accuracy: 0.9994
Test accuracy: 0.9993830919265747


### Predictions:

In [3]:
# Load and preprocess a single image for prediction
image_path = 'ocular_recognition/test/1141/1141_l_1.png'
img = image.load_img(image_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize pixel values to the range [0, 1]

# Make predictions
# As the model expects two inputs, we need to pass the class labels as well.
# Since we have 300 classes and the true label of the image is unknown, we can create a dummy label.
dummy_label = np.zeros((1, 300))
predictions = model.predict([img_array, dummy_label])

# Get the class label with the highest probability
predicted_class = np.argmax(predictions[0])

# Get the class labels
class_labels = list(test_generator.generator.class_indices.keys())

# Get the predicted class label
predicted_class_label = class_labels[predicted_class]

print(f'Predicted class for the single image: {predicted_class_label}')

1/1 [==============================] - 0s 440ms/step
Predicted class for the single image: 1141


### Example Library Usage:

In [4]:
from AB_Rob_Ocular.AB_Rob_Ocular_Tensor import AdversarialEvaluationMetrics
from AB_Rob_Ocular import OcularAdversarialAttacks as OAA

# Attack method name
attack_name = 'fgsm'

# Create an instance of AdversarialEvaluationMetrics
metrics = AdversarialEvaluationMetrics(model, OAA.fgsm_attack_face, attack_name, num_classes=300, image_format=".png")

# Directory containing the test images
test_dir = 'UFPR_Final/test'

# Directory to save the adversarial images
adv_dir = 'Adversarial_Images/FGSM/'

# Compute the attack success rate and save the adversarial images
attack_success_rate = metrics.compute_attack_success_rate(test_dir, epsilon=0.001, adv_dir=adv_dir, save_image=True)

1/1 [==============================] - 0s 24ms/step


In [5]:
print(f"The attack success rate is {attack_success_rate}")

The attack success rate is 0.14682294879703886
